# test for vertex ai connection

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel

# Set proxy if needed
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPs_PROXY'] = 'http://127.0.0.1:10809'

# TODO(developer): Update and un-comment below line
project_id = "log-analysis-433902"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

response = model.generate_content(
    "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"
)

print(response.text)

# test for regular expression

In [ ]:
import re

In [ ]:
text = '''这里是文本，其中包含URL和IP地址，例如：http://www.example.com和 86.193.48.194。
Jul 18 03:26:49 combo ftpd[25648]: connection from 211.72.151.162 () at Mon Jul 18 03:26:49 2005
ruser= rhost=mail.xmjl.com  user=root
ruser= rhost=212-41-230-229.hebragasse.xdsl-line.inode.at
ruser= rhost=pppzss.shenzhen.gd.cn
ruser= rhost=wsip-24-120-168-221.lv.lv.cox.net'''

text = '''62.99.164.82.sh.interxion.inode.at'''

In [ ]:

url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
rhost_pattern = r'([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'
# rhost_pattern = r'([a-zA-Z0-9.-]+\.[a-zA-Z]{2,}|[0-9-]+(?:\.[0-9]+){3})'
ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'


In [ ]:

urls = re.findall(url_pattern, text)
rhost = re.findall(rhost_pattern, text)
ips = re.findall(ip_pattern, text)

print("URLs:", urls)
print("IPs:", ips)
print("RHOST:", rhost)

# test for read log file

In [3]:
with open('data/Linux.txt', 'rb', encoding="utf-8") as f:
    text = f.read()
    print(text)

position = 70072
with open('data/Linux.txt', 'rb') as f:
    for i in range(position-1000, position+1000):
        f.seek(i)  # 跳转到指定位置
        byte = f.read(1)  # 读取一个字节
        print(f"{byte.decode('utf-8')}",end="")


ValueError: binary mode doesn't take an encoding argument

# test for json schema of vertex ai

In [ ]:
import vertexai

from vertexai.generative_models import GenerationConfig, GenerativeModel

# Set proxy if needed
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPs_PROXY'] = 'http://127.0.0.1:10809'

# TODO(developer): Update and un-comment below line
project_id = "log-extraction"
vertexai.init(project=project_id, location="us-central1")

In [ ]:

# response_schema = {
#     "type": "ARRAY",
#     "items": {
#         "type": "OBJECT",
#         "properties": {
#             "Item type": {
#                 "type": "STRING",
#                 "enum": ["IP", "URL", "Memory"],
#             },
#             "Item content": {"type": "STRING"},
#         },
#         "required": ["Item type", "Item content"],
#     },
# }
response_schema = {
    "type": "OBJECT",
    "properties": {
        "url": {
            "type": "ARRAY", 
            "items": {"type": "STRING"},
            },
        "IP": {
            "type": "ARRAY", 
            "items": {"type": "STRING"}, 
            },
        "memory": {
            "type": "ARRAY", 
            "items": {"type": "STRING"},
            },
    },
    "required": ["url", "IP", "memory"],
}


In [ ]:
prompt = """Given a document, your task is to extract the text value of the following entities: {url(domain)}, {IP}, {compuation resources(memory, cpu)}
- The contents must only include text strings found in the document.
- The contents only apear in response one time. Please list them in different catagories.
Feb 19 11:29:58 combo sshd(pam_unix)[5386]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=222.168.14.22 
Feb 19 21:28:43 combo sshd(pam_unix)[5447]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=211.96.97.37  user=root
Feb 20 07:36:03 combo ftpd[5647]: connection from 219.145.93.74 () at Mon Feb 20 07:36:03 2006 

Feb 20 12:14:13 combo unix_chkpwd[5702]: check pass; user unknown
Feb 20 12:14:13 combo sshd(pam_unix)[5688]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=vedgi-gw.rosprint.net 

Feb 20 12:14:15 combo unix_chkpwd[5706]: check pass; user unknown
Feb 20 12:14:15 combo sshd(pam_unix)[5690]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=vedgi-gw.rosprint.net 

Feb 22 01:55:44 combo ftpd[5772]: connection from 210.212.240.242 () at Wed Feb 22 01:55:44 2006 

Feb 22 11:46:37 combo sshd(pam_unix)[5989]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=203186089173.ctinets.com  user=root

Feb 22 13:26:34 combo sshd(pam_unix)[6028]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220.194.58.127  user=root

Feb 22 23:41:37 combo ftpd[6043]: connection from 24.229.9.65 (24.229.9.65.res-cmts.sm.ptd.net) at Wed Feb 22 23:41:37 2006 

Feb 23 12:02:26 combo sshd(pam_unix)[6064]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=80.34.110.175  user=fax
Feb 23 12:02:26 combo sshd(pam_unix)[6070]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=175.red-80-34-110.staticip.rima-tde.net  user=fax
Feb 23 12:02:26 combo sshd(pam_unix)[6072]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=175.red-80-34-110.staticip.rima-tde.net  user=fax
Feb 23 12:02:26 combo sshd(pam_unix)[6066]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=175.red-80-34-110.staticip.rima-tde.net  user=fax
Feb 23 12:02:26 combo sshd(pam_unix)[6067]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=175.red-80-34-110.staticip.rima-tde.net  user=fax
Feb 23 12:02:26 combo sshd(pam_unix)[6073]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=175.red-80-34-110.staticip.rima-tde.net  user=fax
Feb 23 13:54:46 combo unix_chkpwd[6096]: check pass; user unknown
Feb 23 13:54:46 combo sshd(pam_unix)[6076]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=66.50.123.66 
"""

In [ ]:

model = GenerativeModel("gemini-1.5-pro-001")

response = model.generate_content(
    prompt,
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)

print(response.text)

#  pydantic for json validation

In [1]:
from typing import List

from pydantic import BaseModel


class Extract(BaseModel):
    IP: list[str]
    URL: list[str]
    memory: list[str]

l = Extract(IP=['192.168.1.1', '192.168.1.2'], URL=['http://www.example.com', 'http://www.example.org'], memory=['100', '200'])
print(l.IP)


['192.168.1.1', '192.168.1.2']
